<a href="https://colab.research.google.com/github/krishna324-art/LLM-from-Scratch-/blob/main/Datapreparation_and_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open("/content/the-verdict.txt","r",encoding="utf-8")as f:
  raw_text=f.read()
print("Total Number of characters:",len(raw_text))
print(raw_text[:99])

Total Number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
#goal is to tokenize this 20479
import re
text= "Hello, world. This, is a test."
result=re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
#to split commas and periods
results=re.split(r'([,.]|\s)',text)
print(results)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
#to remove white space characters
results1=[item for item in results if item.strip()]
print(results1)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
text="Hello, world. Is this-- a test?"
results= re.split(r'([,.:;?_!"()\']|--|\s)',text)
results=[item.strip()for item in results if item.strip()]
print(results)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [ ]:
#strip out whitespace from each item and then filter out any empty string
result=[item for item in results if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [ ]:
preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
print(len(preprocessed))

4690


In [ ]:
#creating token id
all_words=sorted(set(preprocessed))
vocab_size= len(all_words)
print(vocab_size)#vocab size has all unique words only

1130


In [ ]:
vocab={token:integer for integer,token in enumerate(all_words)}
for i ,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [ ]:
class Simpletokenizerv1:
  def __init__(self,vocab):
    self.str_to_int= vocab
    self.int_to_str={i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed= [item.strip() for item in preprocessed if item.strip()]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    #replace spaces before the specified punctuations
    text=re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text


In [ ]:
tokenizer= Simpletokenizerv1(vocab)
text= """"It's the last he painted, you know,"
         Mrs. Gisburn said with pardonable pride."""

ids= tokenizer.encode(text)
print(ids)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [ ]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
#add a special context tokens
all_tokens= sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab={token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

1132

In [ ]:
# Printing the last 5 items in the vocab dictionary
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
class SimpletokenizerV2:
 def __init__(self,vocab):
    self.str_to_int= vocab
    self.int_to_str={i:s for s,i in vocab.items()}

 def encode(self,text):
    preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed= [item.strip() for item in preprocessed if item.strip()]
    preprocessed=[
        item if item in self.str_to_int
        else "<|unk|>" for item in preprocessed
    ]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

 def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    #replace spaces before the specified punctuations
    text=re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text



In [ ]:
tokenizer=SimpletokenizerV2(vocab)
text1="Hello, do you like tea?"
text2="In the sunlit terraces of the palace."
text=" <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [ ]:
#BPE
!pip install tiktoken

In [ ]:
import importlib
import tiktoken
print("tiktoken version:",importlib.metadata.version("tiktoken"))

tiktoken version: 0.11.0


In [ ]:
tokenizer= tiktoken.get_encoding("gpt2")

In [ ]:
text=(
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers= tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
strings= tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [ ]:
integrs= tokenizer.encode("Akwirw ier")
print(integrs)
strings= tokenizer.decode(integrs)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [ ]:
#input target pairs
with open("/content/the-verdict.txt","r",encoding="utf-8")as f:
  raw_text=f.read()
enc_text= tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_sample=enc_text[50:]

In [ ]:
context_size=4 #length of the input
x= enc_sample[:context_size]
y=enc_sample[1:context_size+1]
print(f"x:{x}")
print(f"y:{y}")

x:[290, 4920, 2241, 287]
y:[4920, 2241, 287, 257]


In [ ]:
#processing the inputs along with the targets
for i in range(1,context_size+1):
  context= enc_sample[:i]
  desired= enc_sample[i]
  print(context,"---->",desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [ ]:
for i in range(1,context_size+1):
  context= enc_sample[:i]
  desired= enc_sample[i]
  print(tokenizer.decode(context),"---->",tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [ ]:
#implementing data loader
import torch
from torch.utils.data import Dataset, DataLoader
class GPTdatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids=[]
    self.target_ids=[]

    #tokenize the entire text
    token_ids=tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

    #use a sliding window to chunk the book into overlapping sequence of max_legth
    for i in range(0,len(token_ids)- max_length,stride):
      input_chunk=token_ids[i:i+max_length]
      target_chunk= token_ids[i+1: i+max_length +1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt,batch_size=4, max_length=256,
                         stride=128,shuffle=True,drop_last=True,
                         num_workers=0):
  #initialize the tokenizer
  tokenizer= tiktoken.get_encoding("gpt2")

  #craete dataset
  dataset=GPTdatasetV1(txt,tokenizer,max_length,stride)
  #create daataloader
  dataloader=DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers)
  return dataloader


In [ ]:
with open("/content/the-verdict.txt","r",encoding="utf-8")as f:
  raw_text=f.read()

In [ ]:
import torch
print("Pytorch version:",torch.__version__)
dataloader= create_dataloader_v1(raw_text,batch_size=1,max_length=4,stride=1,shuffle=False)
data_iter= iter(dataloader)
first_batch=next(data_iter)
print(first_batch)

Pytorch version: 2.8.0+cu126
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
dataloader= create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=4)
data_iter=iter(dataloader)
inputs,targets= next(data_iter)
print("Inputs:\n",inputs)
print("targets:\n",targets)

Inputs:
 tensor([[34537,   526,   198,   198],
        [  423,  1760,   257,  1049],
        [   11,   314,  1422,   470],
        [  286,   262, 33125,    12],
        [19242,   339,   442, 17758],
        [ 6164,    11,  8104,   736],
        [ 3896,    11,   262,   661],
        [  587,  1498,   284,   910]])
targets:
 tensor([[  526,   198,   198,     1],
        [ 1760,   257,  1049,  1517],
        [  314,  1422,   470,  1337],
        [  262, 33125,    12,   565],
        [  339,   442, 17758,   465],
        [   11,  8104,   736,   465],
        [   11,   262,   661,   508],
        [ 1498,   284,   910,   644]])


In [ ]:
vocab_size=50257
output_dim=256
token_embedding_layer=torch.nn.Embedding(vocab_size,output_dim)

In [ ]:
#data loader
#8-batch size,4 context length, 256 dimentional vector
#8*4*256 tensor
max_length=4
dataloader=create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,
                                stride=max_length,shuffle=False)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)

In [ ]:
print("token Id:\n",inputs)
print("\nInputs shape:\n",inputs.shape)

token Id:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
token_embeddings=token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
context_length=max_length
pos_embedding_layer= torch.nn.Embedding(context_length,output_dim)

In [ ]:
pos_embedding=pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)

torch.Size([4, 256])
